# Classify and Aggregate Approach

This method will use majority result from multple models

Include models;
* ProtCNN
* FastAI Tabular learner 
* RF 
* XGBoost
* SVM
* CNN input 1900 + train on 30 AAs

The possible methods;
* cutoff 3 models
* for <=30 at least two models (ProtCNN and ...) say AMP (0), so it will be AMP
* ..... 

In [3]:
! conda install -c conda-forge keras==2.3.1 -y

/ 

In [1]:
#Use only CPU
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc
from xgboost import XGBClassifier

import pickle5 as pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:


if True:
    with tf.device('/cpu:0'):
        Custom_CNN_RMS  = load_model("/mnt/vdb/thesis/CustomCNN.RMSprop.512_1211.hdf5")
    with tf.device('/cpu:0'):
        Custom_CNN_Adam  = load_model("/mnt/vdb/thesis/CustomCNN.Adam.512_1211.hdf5")
    with tf.device('/cpu:0'):    
        DNN = load_model("../Models/best_DNN_model.hdf5", compile=False)
    with tf.device('/cpu:0'):
        ProtCNN= load_model("/mnt/vdb/ProtCNN.bestmodel.h5") # ProtCNN.bestmodel.h5 CNN256 * 2 - NN 1000  / ProtCNN.512_1211X4.best.hdf5 / ProtCNN.1211.hdf5 CNN128 *2-1211
    with tf.device('/cpu:0'):
        CNN_shortAA = load_model("/mnt/vdb/thesis/best_CNN.1900._30.hdf5")
else:
    Custom_CNN  = load_model("/mnt/vdb/thesis/CustomCNN.512_1211.hdf5")
    DNN = load_model("../Models/best_DNN_model.hdf5", compile=False)
    ProtCNN= load_model("/mnt/vdb/ProtCNN.bestmodel.h5")
    CNN_shortAA = load_model("/mnt/vdb/thesis/best_CNN.1900._30.hdf5")    
    
# Load model 
XGBmodel = XGBClassifier()
XGBmodel.load_model("../Models/best_XGBmodel.model")

with open("../Models/best_RF.model", 'rb') as file:
    RF = pickle.load(file)
    
with open("/mnt/vdb/thesis/best_SVM.rbf.model", 'rb') as file:
    SVM = pickle.load(file)



In [4]:
from fastai.tabular.all import *
from fastai.basics import *
from fastai import *

FASTAI_learner = load_learner( "../Models/TubularLearner.II._200.retrain")

In [5]:
# Test Function for RF, SVM, XGBoost

def predict_withModel(_df ,condition, model):

    if condition == "no_con":
        df = _df.copy()
    elif condition == "<=30" :
        df = _df.loc[_df.length <= 30 ].copy()
    elif condition == ">30" :
        df = _df.loc[_df.length > 30 ].copy()
    elif condition == ">50" :
        df = _df.loc[_df.length > 50 ].copy()
    elif condition == ">70" :
        df = _df.loc[_df.length > 70 ].copy()
    elif condition == ">100" :
        df = _df.loc[_df.length > 100 ].copy()
    elif condition == ">200" :
        df = _df.loc[_df.length > 200 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy()
   
        
        # select records with lenght > 
    #print(df.columns)
    X= np.array(df['reps'].to_list())
    _y = model.predict(X)
    #y_probas = learner.predict(X_test)
    #threshold = 0.5
    #_y = np.where(y_probas > threshold, 1, 0)
    return _y


In [6]:
# Test Function for DNN
def predict_withDNN(_df ,condition, model):

    if condition == "no_con":
        df = _df.copy()
    elif condition == "<=30" :
        df = _df.loc[_df.length <= 30 ].copy()
    elif condition == ">30" :
        df = _df.loc[_df.length > 30 ].copy()
    elif condition == ">50" :
        df = _df.loc[_df.length > 50 ].copy()
    elif condition == ">70" :
        df = _df.loc[_df.length > 70 ].copy()
    elif condition == ">100" :
        df = _df.loc[_df.length > 100 ].copy()
    elif condition == ">200" :
        df = _df.loc[_df.length > 200 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy()
   
     #print(df.columns)
    X= np.array(df['reps'].to_list())
    y_probas = model.predict(X)
    threshold = 0.5
    _y = np.where(y_probas > threshold, 1, 0)

    return _y

In [7]:
# Test Function for ProtCNN, CNN_shortAA

def predict_CNN(_df ,condition, model):
    
    if condition == "no_con":
        df = _df.copy()
    elif condition == "<=30" :
        df = _df.loc[_df.length <= 30 ].copy()
    elif condition == ">30" :
        df = _df.loc[_df.length > 30 ].copy()
    elif condition == ">50" :
        df = _df.loc[_df.length > 50 ].copy()
    elif condition == ">70" :
        df = _df.loc[_df.length > 70 ].copy()
    elif condition == ">100" :
        df = _df.loc[_df.length > 100 ].copy()
    elif condition == ">200" :
        df = _df.loc[_df.length > 200 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy()
        

    X= np.array(df['reps'].to_list())
    X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
    y_probas = model.predict(X_test)
    threshold = 0.5
    _y = np.where(y_probas > threshold, 1, 0)
    
    return _y

In [8]:
# Test Function for FastAI
def predict_FastAI(_df ,condition, model):
    if condition == "no_con":
        df = _df.copy()
    elif condition == "<=30" :
        df = _df.loc[_df.length <= 30 ].copy()
    elif condition == ">30" :
        df = _df.loc[_df.length > 30 ].copy()
    elif condition == ">50" :
        df = _df.loc[_df.length > 50 ].copy()
    elif condition == ">70" :
        df = _df.loc[_df.length > 70 ].copy()
    elif condition == ">100" :
        df = _df.loc[_df.length > 100 ].copy()
    elif condition == ">200" :
        df = _df.loc[_df.length > 200 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy()
        
    #print(df.columns)
    ready_df = df[[ "reps"]]
    df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
    df_new.columns = df_new.columns.astype(str)
    dl = model.dls.test_dl(df_new)
    _preds,_none ,_y = model.get_preds(dl=dl, with_decoded=True)

    return _y



In [9]:
# Aggregate Result Function 
def aggregate_result(row_series, condition):
    freq = row_series.value_counts().to_dict()
    
    _nonAMP = 0
    _AMP = 0
    _final_return = 0
    cutoff_threshold = 3 # RMSpop, Adam , ProtCNN, AmPEP30
    #print(freq)
    if 0 in  freq.keys():
        _AMP = freq[0]

    if 1 in freq.keys():
        _nonAMP = freq[1]  

    #print("AMP " + str(_AMP))
   # print("non AMP " + str(_nonAMP))
     # condition by pass rule if two models say  nonAMP
    #if(condition == "<=30"):
    if _nonAMP >= cutoff_threshold:
        #print("Rule fire")
        return 1 # NonAMP
    
    if _AMP > _nonAMP: # AMP
        _final_return = 0
    elif _AMP < _nonAMP: # NonAMP
        _final_return = 1
    else: #eql 
        pass
    
    return _final_return

In [10]:
# Load Testset
test_dict ={ "shuff1": "../datasets/truthset/AMPs_truthset_42.reps.plk",
            "shuff2": "../datasets/truthset/AMPs_truthset_43.reps.plk",
           "shuff3": "../datasets/truthset/AMPs_truthset_44.reps.plk",
           "shuff4": "../datasets/truthset/AMPs_truthset_45.reps.plk",
           "shuff5": "../datasets/truthset/AMPs_truthset_46.reps.plk",
           "shuff6": "../datasets/truthset/AMPs_truthset_47.reps.plk",
           "shuff7": "../datasets/truthset/AMPs_truthset_48.reps.plk",
           "shuff8": "../datasets/truthset/AMPs_truthset_49.reps.plk",
           "shuff9": "../datasets/truthset/AMPs_truthset_50.reps.plk",
           "shuff10": "../datasets/truthset/AMPs_truthset_51.reps.plk",}

conditions = ["no_con",">200",">100",">70",">50",">30","<=30"]

In [11]:
# Run 
#predict_result_df = result_df = pd.DataFrame( columns=["SVM","RF","XGBoost","ProtCNN","CNN_shortAA","DNN","CustomCNN_Adam","CustomCNN_RMSpop","FastAI"])

aggregate_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","<=30"],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1

for cond in conditions:
    for index, path in test_dict.items():
        predict_result_df = pd.DataFrame()
        df_test = pd.read_pickle(path)
        
        predict_result_df["SVM"] = predict_withModel(df_test, cond, SVM)
        predict_result_df["RF"] = predict_withModel(df_test, cond, RF)
        predict_result_df["XGBoost"] = predict_withModel(df_test, cond, XGBmodel)
        predict_result_df["FastAI"] = predict_FastAI(df_test, cond, FASTAI_learner)
        
        predict_result_df["ProtCNN"] = predict_CNN(df_test, cond, ProtCNN)
        predict_result_df["CNN_shortAA"] = predict_CNN(df_test, cond, CNN_shortAA)
        predict_result_df["DNN"] = predict_withDNN(df_test, cond, DNN)
        
        predict_result_df["CustomCNN_Adam"] = predict_withDNN(df_test, cond, Custom_CNN_Adam)
        predict_result_df["CustomCNN_RMSpop"] = predict_withDNN(df_test, cond, Custom_CNN_RMS)
        predict_result_df['Final'] = predict_result_df.apply(lambda row: aggregate_result(row,cond) , axis = 1) 
        # count 1 only 
        total_non_amp = len(predict_result_df[predict_result_df["Final"] == 1])
        aggregate_df.loc[index,cond] = total_non_amp

In [12]:
aggregate_df

,no_con,>200,>100,>70,>50,>30,<=30
shuff1,30,4,7,16,18,26,4
shuff2,39,4,7,18,29,35,4
shuff3,31,4,6,15,18,25,6
shuff4,24,4,7,12,13,18,6
shuff5,30,4,7,13,16,25,5
shuff6,35,4,7,15,19,27,8
shuff7,30,4,7,14,19,24,6
shuff8,31,4,7,13,23,28,3
shuff9,42,4,7,16,29,36,6
shuff10,27,4,7,14,15,21,6
